In [1]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model
import coralme

In [2]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [3]:
# !bash report.sh
organisms = set(pandas.read_csv("biomass_constrained.txt",index_col=0,header=None).index.to_list())
done = {i.split(".csv")[0] for i in os.listdir("./cases/fluxes/low_iron-context/")}

In [4]:
len(organisms)

495

### Load

In [5]:
def load_data(path):
    try:
        df = pandas.read_csv(path,index_col=0)[["fluxes"]]
    except:
        df = pandas.DataFrame(columns=["fluxes"])
    df.columns = [org]
    return df

In [14]:
from tqdm import tqdm
conditions = ["base","low_iron","low_zinc","high_FA","high_carbs","high_prot"]
# conditions = ["base","low_iron","high_carbs","high_FA"]

In [15]:
overflow = {i:pandas.DataFrame() for i in conditions}
growth = {i:{} for i in conditions}

In [16]:
for org in tqdm(organisms):
    for c in conditions:
        if c == "base":
            tmp = load_data("./cases/fluxes/IBD/{}.csv".format(org))
        else:
            tmp = load_data("./cases/fluxes/{}-context/{}.csv".format(c,org))
        if tmp.empty:
            val = None
        else:
            val = tmp[org]["biomass_dilution"]
        growth[c][org] = val
growth = pandas.DataFrame.from_dict(growth)

100%|██████████| 495/495 [01:04<00:00,  7.67it/s]


### Save

In [17]:
growth.sort_index().to_csv("datasets/all_growth_rates-context.csv")